# Haiti Health Data Analysis

## Importing Packages

In [1]:
import sqlalchemy 
import numpy as np
import pandas as pd

import pyodbc

## Loading The Datasets

In [2]:
covid_cases = pd.read_csv("./datasets/mspp_covid19_cases.csv", index_col=0)
spa = pd.read_csv("./datasets/spa.csv",index_col=0)
departement = pd.read_excel("./datasets/hti_adminboundaries_tabulardata.xlsx",sheet_name=1,index_col=0)
commune  = pd.read_excel("./datasets/hti_adminboundaries_tabulardata.xlsx",sheet_name=2,index_col=0)

## Data Wrangling

### Cleaning the Covid cases dataset

In [3]:
covid_cases.head()

,departement,cas_suspects,cas_confirmes,deces,taux_de_letalite,document_date
0,Artibonite,47,4,0,0.0,2020-04-15
1,Centre,14,1,0,0.0,2020-04-15
2,Grande'Anse,2,0,0,0.0,2020-04-15
3,Nippes,16,2,0,0.0,2020-04-15
4,Nord,28,0,0,0.0,2020-04-15


In [4]:
covid_cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 0 to 98
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   departement       99 non-null     object 
 1   cas_suspects      99 non-null     int64  
 2   cas_confirmes     99 non-null     int64  
 3   deces             99 non-null     int64  
 4   taux_de_letalite  99 non-null     float64
 5   document_date     99 non-null     object 
dtypes: float64(1), int64(3), object(2)
memory usage: 5.4+ KB


#### Dropping Total rows

In [5]:
covid_cases=covid_cases.drop([10,21,32,43,54,65,76,87,98])

In [6]:
covid_cases['departement'].value_counts()

Nippes         9
Sud            9
Nord-Est       9
Sud-Est        9
Centre         9
Grande'Anse    9
Artibonite     9
Nord-Ouest     9
Nord           9
Ouest          8
Guest          1
Name: departement, dtype: int64

#### Fixing Guest Typo

In [7]:
covid_cases=covid_cases.sort_values(by=['document_date','departement'],ascending=[True,True])
covid_cases = covid_cases.drop_duplicates()
covid_cases['departement'] = covid_cases['departement'].replace({'Guest' :'Ouest' }, inplace =False)



In [8]:
covid_cases['departement'].value_counts()

Nippes         9
Sud            9
Nord-Est       9
Sud-Est        9
Centre         9
Ouest          9
Grande'Anse    9
Artibonite     9
Nord-Ouest     9
Nord           9
Name: departement, dtype: int64

In [9]:
covid_cases1 = pd.merge(covid_cases,departement,how= 'left', left_on='departement', right_on='adm1_fr')

covid_cases1= covid_cases1.iloc[:,[1,2,3,4,5,9]]
covid_cases1

,cas_suspects,cas_confirmes,deces,taux_de_letalite,document_date,adm1code
0,12,1,0,0.00,2020-04-07,HT05
1,11,0,0,0.00,2020-04-07,HT06
2,1,0,0,0.00,2020-04-07,HT08
3,14,2,0,0.00,2020-04-07,HT10
4,19,0,0,0.00,2020-04-07,HT03
...,...,...,...,...,...,...
85,24,8,2,0.25,2020-04-16,HT04
86,7,1,0,0.00,2020-04-16,HT09
87,325,24,1,0.04,2020-04-16,HT01
88,8,0,0,0.00,2020-04-16,HT07


In [10]:
covid_cases1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 0 to 89
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cas_suspects      90 non-null     int64  
 1   cas_confirmes     90 non-null     int64  
 2   deces             90 non-null     int64  
 3   taux_de_letalite  90 non-null     float64
 4   document_date     90 non-null     object 
 5   adm1code          90 non-null     object 
dtypes: float64(1), int64(3), object(2)
memory usage: 4.9+ KB


### Cleaning Departement datasets

In [11]:
departement.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, HT to HT
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   adm0_en                 10 non-null     object 
 1   adm0_fr                 10 non-null     object 
 2   adm0_ht                 10 non-null     object 
 3   adm1code                10 non-null     object 
 4   adm1_en                 10 non-null     object 
 5   adm1_fr                 10 non-null     object 
 6   adm1_ht                 10 non-null     object 
 7   IHSI_UNFPA_2019         10 non-null     float64
 8   IHSI_UNFPA_2019_female  10 non-null     int64  
 9   IHSI_UNFPA_2019_male    10 non-null     int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 880.0+ bytes


In [12]:
departement=departement.loc[:,['adm1code','adm1_en','IHSI_UNFPA_2019','IHSI_UNFPA_2019_female','IHSI_UNFPA_2019_male']]

In [13]:
departement.head()

,adm1code,adm1_en,IHSI_UNFPA_2019,IHSI_UNFPA_2019_female,IHSI_UNFPA_2019_male
adm0code,,,,,
HT,HT01,West,5.010206e+06,2587360,2422846
HT,HT02,South-East,8.939007e+05,448111,445790
HT,HT03,North,1.159762e+06,589825,569932
HT,HT04,North-East,4.444631e+05,221501,222966
HT,HT05,Artibonite,2.047153e+06,1034065,1013085


### Cleaning the commune Dataset

In [14]:
commune.head()

,adm0_en,adm0_fr,adm0_ht,adm1code,adm1_en,adm1_fr,adm1_ht,adm2code,adm2_en,adm2_fr,adm2_ht,IHSI_UNFPA_2019,IHSI_UNFPA_2019_female,IHSI_UNFPA_2019_male
adm0code,,,,,,,,,,,,,,
HT,Haiti,Haïti,Ayiti,HT01,West,Ouest,Lwès,HT0111,Port-au-Prince,Port-au-Prince,,1.227540e+06,648353,579188
HT,Haiti,Haïti,Ayiti,HT01,West,Ouest,Lwès,HT0112,Delmas,Delmas,,4.914342e+05,258526,232908
HT,Haiti,Haïti,Ayiti,HT01,West,Ouest,Lwès,HT0113,Carrefour,Carrefour,,6.357646e+05,333500,302263
HT,Haiti,Haïti,Ayiti,HT01,West,Ouest,Lwès,HT0114,Petion-Ville,Pétion-Ville,,4.685251e+05,241407,227118
HT,Haiti,Haïti,Ayiti,HT01,West,Ouest,Lwès,HT0115,Kenscoff,Kenscoff,,7.140848e+04,35608,35800


In [15]:
commune=commune.loc[:,['adm1code','adm1_en','adm2code','adm2_en','IHSI_UNFPA_2019','IHSI_UNFPA_2019_female','IHSI_UNFPA_2019_male']]

In [16]:
commune.head()

,adm1code,adm1_en,adm2code,adm2_en,IHSI_UNFPA_2019,IHSI_UNFPA_2019_female,IHSI_UNFPA_2019_male
adm0code,,,,,,,
HT,HT01,West,HT0111,Port-au-Prince,1.227540e+06,648353,579188
HT,HT01,West,HT0112,Delmas,4.914342e+05,258526,232908
HT,HT01,West,HT0113,Carrefour,6.357646e+05,333500,302263
HT,HT01,West,HT0114,Petion-Ville,4.685251e+05,241407,227118
HT,HT01,West,HT0115,Kenscoff,7.140848e+04,35608,35800


### Cleaning the spa Dataset

In [17]:
spa.head()

,index,facil,depart,departn,vilcom,vilcomn,factype,facdesc_1,facdesc,mga,service_laboratory,num_beds,ambulance,full_time
0,0,1,1,Ouest,11,Port-Au-Prince,7.0,DISPENSAIRE,DISPENSAIRE,3,1.0,NaN,3.0,5.0
1,1,2,1,Ouest,11,Port-Au-Prince,3.0,HOPITAL,HOPITAL COMMUNAUTAIRE DE REFERENCE,1,1.0,25.0,3.0,18.0
2,2,3,1,Ouest,11,Port-Au-Prince,6.0,CENTRE DE SANTE SANS LIT,CENTRE DE SANTE SANS LIT,3,1.0,201.0,1.0,141.0
3,3,4,1,Ouest,11,Port-Au-Prince,5.0,CENTRE DE SANTE AVEC LIT,CENTRE DE SANTE AVEC LIT,3,1.0,7.0,3.0,13.0
4,4,5,1,Ouest,11,Port-Au-Prince,6.0,CENTRE DE SANTE SANS LIT,CENTRE DE SANTE SANS LIT,2,1.0,NaN,3.0,10.0


#### Cleaning vilcomn column

In [18]:
  spa['vilcomn']= spa['vilcomn'].replace({"L!og!ne":"Leogane","Limb!":"Limbe","St-Ls-du-Nord'Ouest"	:"Saint-Louis du Nord" ,"Cayes" :"Les Cayes" }, inplace =False)

In [19]:

x = spa['vilcomn'].drop_duplicates()
display(x)

0       Port-Au-Prince
23      Port-au-Prince
48         Cit!-Soleil
59              Delmas
91             Tabarre
             ...      
1011           St Marc
1021        Gros Morne
1023      Port-De-Paix
1025    Port Au Prince
1031         Mirbalais
Name: vilcomn, Length: 151, dtype: object

In [20]:
b = commune.loc[:,'adm2_en'].astype('str')
b = b.reset_index() 
b.sort_values(by ="adm2_en" )

,adm0code,adm2_en
108,HT,Abricots
33,HT,Acul du Nord
67,HT,Anse Rouge
18,HT,Anse a Galets
112,HT,Anse d'Hainault
...,...,...
106,HT,Tiburon
90,HT,Torbeck
55,HT,Trou du Nord
59,HT,Vallieres


In [21]:
!pip3 install rapidfuzz

In [22]:
from rapidfuzz import process
t = spa['vilcomn'].apply(lambda x: process.extractOne(x , b.iloc[:,1]))
t

0                (Port-au-Prince, 100.0, 0)
1                (Port-au-Prince, 100.0, 0)
2                (Port-au-Prince, 100.0, 0)
3                (Port-au-Prince, 100.0, 0)
4                (Port-au-Prince, 100.0, 0)
                       ...                 
1028                    (Leogane, 100.0, 8)
1029                    (Jacmel, 100.0, 20)
1030                  (Arcahaie, 100.0, 16)
1031    (Mirebalais, 94.73684210526315, 81)
1032                    (Jacmel, 100.0, 20)
Name: vilcomn, Length: 1033, dtype: object

In [23]:
def listed( t) : 
    list1 = []
    a=0
    for i in range(0,t.shape[0]) :
        a = list(t[i])
        list1.append(a[0]) 
    spa['new'] = list1    
    return spa['new']             

In [24]:
listed(t)

0       Port-au-Prince
1       Port-au-Prince
2       Port-au-Prince
3       Port-au-Prince
4       Port-au-Prince
             ...      
1028           Leogane
1029            Jacmel
1030          Arcahaie
1031        Mirebalais
1032            Jacmel
Name: new, Length: 1033, dtype: object

### MERGING spa and commune


In [25]:
f_merge=pd.merge(left=spa, right=commune, how='left', left_on='new', right_on='adm2_en')

In [26]:
f_merge.head()

,index,facil,depart,departn,vilcom,vilcomn,factype,facdesc_1,facdesc,mga,...,ambulance,full_time,new,adm1code,adm1_en,adm2code,adm2_en,IHSI_UNFPA_2019,IHSI_UNFPA_2019_female,IHSI_UNFPA_2019_male
0,0,1,1,Ouest,11,Port-Au-Prince,7.0,DISPENSAIRE,DISPENSAIRE,3,...,3.0,5.0,Port-au-Prince,HT01,West,HT0111,Port-au-Prince,1.227540e+06,648353,579188
1,1,2,1,Ouest,11,Port-Au-Prince,3.0,HOPITAL,HOPITAL COMMUNAUTAIRE DE REFERENCE,1,...,3.0,18.0,Port-au-Prince,HT01,West,HT0111,Port-au-Prince,1.227540e+06,648353,579188
2,2,3,1,Ouest,11,Port-Au-Prince,6.0,CENTRE DE SANTE SANS LIT,CENTRE DE SANTE SANS LIT,3,...,1.0,141.0,Port-au-Prince,HT01,West,HT0111,Port-au-Prince,1.227540e+06,648353,579188
3,3,4,1,Ouest,11,Port-Au-Prince,5.0,CENTRE DE SANTE AVEC LIT,CENTRE DE SANTE AVEC LIT,3,...,3.0,13.0,Port-au-Prince,HT01,West,HT0111,Port-au-Prince,1.227540e+06,648353,579188
4,4,5,1,Ouest,11,Port-Au-Prince,6.0,CENTRE DE SANTE SANS LIT,CENTRE DE SANTE SANS LIT,2,...,3.0,10.0,Port-au-Prince,HT01,West,HT0111,Port-au-Prince,1.227540e+06,648353,579188


In [27]:
f_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1033 entries, 0 to 1032
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1033 non-null   int64  
 1   facil                   1033 non-null   int64  
 2   depart                  1033 non-null   int64  
 3   departn                 1033 non-null   object 
 4   vilcom                  1033 non-null   int64  
 5   vilcomn                 1033 non-null   object 
 6   factype                 1033 non-null   float64
 7   facdesc_1               1033 non-null   object 
 8   facdesc                 1033 non-null   object 
 9   mga                     1033 non-null   int64  
 10  service_laboratory      1007 non-null   float64
 11  num_beds                474 non-null    float64
 12  ambulance               1007 non-null   float64
 13  full_time               1007 non-null   float64
 14  new                     1033 non-null   

In [28]:
newtab= spa.loc[:,['factype','facdesc_1','facdesc']]
newtab=newtab.drop_duplicates().sort_values(by='factype')
newtab

,factype,facdesc_1,facdesc
16,1.0,HOPITAL,HOPITAL UNIVERSITAIRE
289,2.0,HOPITAL,HOPITAL DEPARTEMENTAL
1,3.0,HOPITAL,HOPITAL COMMUNAUTAIRE DE REFERENCE
12,4.0,HOPITAL,AUTRES HOPITAUX
3,5.0,CENTRE DE SANTE AVEC LIT,CENTRE DE SANTE AVEC LIT
2,6.0,CENTRE DE SANTE SANS LIT,CENTRE DE SANTE SANS LIT
0,7.0,DISPENSAIRE,DISPENSAIRE


In [29]:
f_mergeclean = f_merge.loc[:,['factype','mga','ambulance','service_laboratory', 'num_beds',
       'ambulance', 'full_time', 'adm1code', 'adm2code']]

## Loading the dataset to SQL SERVER

In [30]:
import urllib
import sqlalchemy
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=localhost;'
                      'Database=Haiti_Health_Data_Analysis;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=localhost;"
                                 "DATABASE=Haiti_Health_Data_Analysis;"
                                 "Trusted_Connection=yes")  

engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))      
with engine.connect() as conn, conn.begin():   
  newtab.to_sql('factype',conn, if_exists='replace', index= True)
  f_mergeclean.to_sql('spa',conn, if_exists='replace', index= True)
  covid_cases1.to_sql('Covid_Cases',conn, if_exists='replace', index= True)
  commune.to_sql('Commune',conn, if_exists='replace', index= False)
  departement.to_sql('Departement',conn, if_exists='replace', index= False)



In [31]:




sql_query = pd.read_sql_query(
"""
SELECT * from spa
  """
,conn)
display(sql_query)
print(type(sql_query))

ResourceClosedError: This Connection is closed

 ### Question 2. Calculate the number of health facilities per commune.

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT commune.adm2_en ,count(spa.factype) AS Number_of_facilities
FROM commune INNER JOIN spa ON commune.adm2code=spa.adm2code 
GROUP BY commune.adm2_en
  """
,conn)
display(sql_query)
print(type(sql_query))


,adm2_en,Number_of_facilities
0,Abricots,2
1,Acul du Nord,8
2,Anse a Galets,13
3,Anse d'Hainault,4
4,Anse Rouge,7
...,...,...
134,Tiburon,1
135,Torbeck,6
136,Trou du Nord,4
137,Vallieres,2


<class 'pandas.core.frame.DataFrame'>


### --* Question 3. Calculate the number of health facilities by commune and by type of health facility.

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT commune.adm2_en, factype.facdesc , count(*)
FROM commune INNER JOIN spa ON commune.adm2code=spa.adm2code INNER JOIN factype ON spa.factype=factype.factype
GROUP BY commune.adm2_en , factype.facdesc
ORDER BY commune.adm2_en ASC
  """
,conn)
display(sql_query)
print(type(sql_query))

,adm2_en,facdesc,
0,Abricots,CENTRE DE SANTE SANS LIT,2
1,Acul du Nord,AUTRES HOPITAUX,1
2,Acul du Nord,CENTRE DE SANTE SANS LIT,4
3,Acul du Nord,DISPENSAIRE,3
4,Anse a Galets,CENTRE DE SANTE AVEC LIT,2
...,...,...,...
366,Trou du Nord,DISPENSAIRE,3
367,Vallieres,DISPENSAIRE,2
368,Verrettes,AUTRES HOPITAUX,2
369,Verrettes,CENTRE DE SANTE SANS LIT,4


<class 'pandas.core.frame.DataFrame'>


### Question 4. Calculate the number of health facilities by municipality and by department.

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT commune.adm2_en, departement.adm1_en, COUNT(*)
FROM commune INNER JOIN spa ON commune.adm2code=spa.adm2code INNER JOIN  departement ON departement.adm1code=commune.adm1code
GROUP BY commune.adm2_en , departement.adm1_en
  """
,conn)
display(sql_query)
print(type(sql_query))

,adm2_en,adm1_en,
0,Anse Rouge,Artibonite,7
1,Desdunes,Artibonite,3
2,Dessalines,Artibonite,21
3,Ennery,Artibonite,7
4,Gonaives,Artibonite,16
...,...,...,...
134,Petit-Goave,West,10
135,Pointe a Raquette,West,7
136,Port-au-Prince,West,64
137,Tabarre,West,19


<class 'pandas.core.frame.DataFrame'>


### Question 5: Calculate the number of sites by type (mga) and by department.

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT [c].[mga], [p].[adm1_en], COUNT(*) as Number FROM [Haiti_Health_Data_Analysis].[dbo].[spa] as c
INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[Departement]  AS p ON c.[adm1code] = p.[adm1code]
GROUP BY p.[adm1_en],[c].[mga]
"""
,conn)
display(sql_query)
print(type(sql_query))

,mga,adm1_en,Number
0,1,Artibonite,59
1,1,Centre,24
2,1,Grande'Anse,25
3,1,Nippes,9
4,1,North,26
5,1,North-East,29
6,1,North-West,53
7,1,South,29
8,1,South-East,33
9,1,West,63


<class 'pandas.core.frame.DataFrame'>


### Question 6: Calculate the number of sites with an ambulance by commune and by department (ambulance = 1.0).

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT  [p].[adm1_en],[t].[adm2_en], COUNT(*) as Number FROM [Haiti_Health_Data_Analysis].[dbo].[spa] as c
INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[Departement]  AS p ON c.[adm1code] = p.[adm1code]
INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[Commune]  AS t ON t.[adm2code] = c.[adm2code]
WHERE [c].[ambulance] = 1
GROUP BY p.[adm1_en],[t].[adm2_en]
ORDER BY [p].[adm1_en]
"""
,conn)
display(sql_query)
print(type(sql_query))

,adm1_en,adm2_en,Number
0,Artibonite,Anse Rouge,8
1,Artibonite,Dessalines,8
2,Artibonite,Ennery,8
3,Artibonite,Gonaives,16
4,Artibonite,Grande Saline,8
...,...,...,...
83,West,Gressier,8
84,West,Petit-Goave,24
85,West,Port-au-Prince,64
86,West,Tabarre,32


<class 'pandas.core.frame.DataFrame'>


###  Question 7. Calculate the number of hospitals per 10k inhabitants by department.

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT * FROM Hopital LEFT JOIN dispensaire on Hopital.Commune=dispensaire.Commune
WHERE Hopital.Num_Of_Hop > dispensaire.Num_Of_Dis
UNION 
SELECT * FROM Hopital RIGHT JOIN dispensaire on Hopital.Commune=dispensaire.Commune
WHERE Hopital.Num_Of_Hop > dispensaire.Num_Of_Dis

  """
,conn)
display(sql_query)
print(type(sql_query))

### Question 8. Calculate the number of sites per 10k inhabitants per department.

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT [adm1_en], ROUND((COUNT(p.factype)*10000/([IHSI_UNFPA_2019])),2) as Number_of_hospitals_10k_habitants  FROM [Haiti_Health_Data_Analysis].[dbo].[Departement] AS c
INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[spa]  AS p ON c.[adm1code] = p.[adm1code]
INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[Factype]  AS t ON [p].[factype] = [t].[factype]
WHERE [t].facdesc_1 = 'HOPITAL'
GROUP BY [adm1_en],[IHSI_UNFPA_2019]

  """
,conn)
display(sql_query)
print(type(sql_query))


,adm1_en,Number_of_hospitals_10k_habitants
0,Grande'Anse,0.67
1,South,1.11
2,South-East,0.45
3,Centre,0.47
4,North-West,0.46
5,West,1.02
6,North,1.24
7,North-East,0.36
8,Artibonite,0.66
9,Nippes,0.68


<class 'pandas.core.frame.DataFrame'>


### Question 9: Calculate the number of beb per 1,000 inhabitants per department.

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT [adm1_en], ROUND((COUNT(p.factype)*10000/([IHSI_UNFPA_2019])),2) as Number_of_hospitals_10k_habitants  FROM [Haiti_Health_Data_Analysis].[dbo].[Departement] AS c
INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[spa]  AS p ON c.[adm1code] = p.[adm1code]
GROUP BY [adm1_en],[IHSI_UNFPA_2019]

  """
,conn)
display(sql_query)
print(type(sql_query))

,adm1_en,Number_of_hospitals_10k_habitants
0,Nippes,3.75
1,North-East,3.69
2,Grande'Anse,4.41
3,South,3.79
4,Centre,2.60
5,North-West,3.45
6,South-East,3.36
7,North,4.21
8,Artibonite,2.97
9,West,2.81


<class 'pandas.core.frame.DataFrame'>


### Question 9: Calculate the number of beb per 1,000 inhabitants per department.

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT [adm1_en], ROUND((SUM([p].[num_beds]*1000/[IHSI_UNFPA_2019])),2) as Number_of_bed_1k_habitants  FROM [Haiti_Health_Data_Analysis].[dbo].[Departement] AS c
INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[spa]  AS p ON c.[adm1code] = p.[adm1code]
GROUP BY [adm1_en],[IHSI_UNFPA_2019]


  """
,conn)
display(sql_query)
print(type(sql_query))

,adm1_en,Number_of_bed_1k_habitants
0,Grande'Anse,2.20
1,South,3.48
2,South-East,1.33
3,Centre,3.21
4,North-West,1.69
5,West,2.87
6,North,3.43
7,North-East,1.87
8,Artibonite,2.12
9,Nippes,1.63


<class 'pandas.core.frame.DataFrame'>


### Question 10. How many communes have fewer dispensaries than hospitals?

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT [adm2_en], factype.facdesc_1 , count(*) AS NUMB_HOP
INTO ##hs
FROM [Haiti_Health_Data_Analysis].[dbo].[Commune] INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[spa] ON commune.adm2code=spa.adm2code INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[Factype]  ON spa.factype=factype.factype
WHERE factype.facdesc_1 = 'HOPITAL'
GROUP BY commune.adm2_en , factype.facdesc_1
ORDER BY commune.adm2_en ASC
  """
,conn)

display(sql_query)
print(type(sql_query))

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT [adm2_en], factype.facdesc_1 , count(*) AS NUMB_HOP
INTO #hd
FROM [Haiti_Health_Data_Analysis].[dbo].[Commune] INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[spa] ON commune.adm2code=spa.adm2code INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[Factype]  ON spa.factype=factype.factype
WHERE factype.facdesc_1 = 'DISPENSAIRE'
GROUP BY commune.adm2_en , factype.facdesc_1
ORDER BY commune.adm2_en ASC
"""
,conn)

display(sql_query)
print(type(sql_query))

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT *  FROM ##hs INNER JOIN ##hd ON ##hd.adm2_en = ##hs.adm2_en 
WHERE ##hs.NUMB_HOP > ##hd.NUMB_DIS
"""
,conn)

display(sql_query)
print(type(sql_query))

,adm2_en,facdesc_1,NUMB_HOP,adm2_en,facdesc_1,NUMB_DIS
0,Bas Limbe,HOPITAL,8,Bas Limbe,DISPENSAIRE,4
1,Cap-Haitien,HOPITAL,20,Cap-Haitien,DISPENSAIRE,8
2,Carrefour,HOPITAL,16,Carrefour,DISPENSAIRE,12
3,Croix-Des-Bouquets,HOPITAL,32,Croix-Des-Bouquets,DISPENSAIRE,4
4,Delmas,HOPITAL,44,Delmas,DISPENSAIRE,4
5,Les Cayes,HOPITAL,20,Les Cayes,DISPENSAIRE,12
6,Petion-Ville,HOPITAL,28,Petion-Ville,DISPENSAIRE,8
7,Plaine du Nord,HOPITAL,20,Plaine du Nord,DISPENSAIRE,12
8,Port-au-Prince,HOPITAL,64,Port-au-Prince,DISPENSAIRE,8


<class 'pandas.core.frame.DataFrame'>


 ### Question 11 How many  Letality rate per month

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT Datename(m,[document_date]) as Month, Sum([taux_de_letalite]) As Letality_Rate From [Haiti_Health_Data_Analysis].[dbo].[Covid_cases]
Group by Datename(m,[document_date])
ORDER BY Datename(m,[document_date])
"""
,conn)

display(sql_query)
print(type(sql_query))

,Month,Letality_Rate
0,April,2.84


<class 'pandas.core.frame.DataFrame'>


### Question 12 How many Death rate per month

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT Datename(m,[document_date]) As Month , Cast(Sum([taux_de_letalite]) as nvarchar(10)) +' %'  As Letality_Rate From [Haiti_Health_Data_Analysis].[dbo].[Covid_cases]
Group by Datename(m,[document_date])
ORDER By Month
"""
,conn)

display(sql_query)
print(type(sql_query))

,Month,Letality_Rate
0,April,0.0315556 %


<class 'pandas.core.frame.DataFrame'>


 ### Question 13 How many Prevalence per month

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT Datename(m,[document_date]) As Month , Cast(Sum([c].[cas_confirmes])/Sum([p].[IHSI_UNFPA_2019]) as nvarchar(100)) +' %'  As Prevalence_per_m From [Haiti_Health_Data_Analysis].[dbo].[Covid_cases] AS c 
INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[Departement] As p ON [c].[adm1code] = [p].adm1code
Group by Datename(m,[document_date])
ORDER By Month
"""
,conn)

display(sql_query)
print(type(sql_query))

,Month,Prevalence_per_m
0,April,2.83521e-006 %


<class 'pandas.core.frame.DataFrame'>


### Question 14 How many Prevalence by department

In [ ]:
sql_query = pd.read_sql_query(
"""
SELECT [p].[adm1_en], Cast([c].[cas_confirmes]/[p].[IHSI_UNFPA_2019] as nvarchar(100)) +' %'  As Prevalence_per_m From [Haiti_Health_Data_Analysis].[dbo].[Covid_cases] AS c 
INNER JOIN [Haiti_Health_Data_Analysis].[dbo].[Departement] As p ON [c].[adm1code] = [p].adm1code
Group by p.[adm1_en],[p].[IHSI_UNFPA_2019],[c].[cas_confirmes]
ORDER By Prevalence_per_m DESC

"""
,conn)

display(sql_query)
print(type(sql_query))

,adm1_en,Prevalence_per_m
0,North-East,6.74972e-006 %
1,Nippes,5.68839e-006 %
2,Artibonite,4.88483e-007 %
3,West,4.79022e-006 %
4,South-East,4.47477e-006 %
5,West,4.39104e-006 %
6,West,4.19144e-006 %
7,West,3.79226e-006 %
8,West,3.59267e-006 %
9,North-East,2.24991e-006 %


<class 'pandas.core.frame.DataFrame'>


In [ ]:
conn.close()